In [ ]:
import pandas as pd
import numpy as np
import whisper
from pytube import YouTube

In [ ]:
model = whisper.load_model('base')

In [ ]:
youtube_video_url = "https://www.youtube.com/watch?v=Q2u0dKRA1cA"
youtube_video = YouTube(youtube_video_url)

In [ ]:
watch_url = youtube_video.watch_url
watch_url

In [ ]:
video_id = youtube_video.video_id
video_id

In [ ]:
stream = youtube_video.streams.filter(only_audio=True).first()
stream

In [ ]:
stream.download(filename=f"./mp4/{video_id}.mp4")

In [ ]:
# do the transcription
output = model.transcribe(f"./mp4/{video_id}.mp4", fp16=False)

In [ ]:
output

In [ ]:
segments = output["segments"]
segments

In [ ]:
df = pd.DataFrame(segments)
df.head(5)

In [ ]:
# Cache the transcription
df.to_csv(f"./csv/{video_id}.csv", index=False)

In [ ]:
df_backup = pd.read_csv(f"./csv/{video_id}.csv")
df_backup.head(3)

In [ ]:
df_export = df_backup[['id', 'start', 'text']]
df_export.head(5)

In [ ]:
df_export.insert(0, 'video_id', video_id)

In [ ]:
df_export.insert(0, 'watch_url', watch_url)

In [ ]:
# Lambda function to modify the URL column
add_time = lambda row: row['watch_url'] + '&t=' + str(int(row['start']))

In [ ]:
df_export.loc[:, 'watch_url'] = df_export.apply(add_time, axis=1)

In [ ]:
df_export

In [ ]:
df_export.to_csv(f"./csv/{video_id}_export.csv", index=False)

In [ ]:
df_embed = pd.read_csv(f"./csv/{video_id}_export.csv")
df_embed.head(3)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
ollama_emb = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [ ]:
# Lambda function to modify the URL column
add_embed = lambda x: ollama_emb.embed_query(x['text'])

In [ ]:
df_embed.loc[:, 'text_embed'] = df_embed.apply(add_embed, axis=1)

In [ ]:
df_embed.head(3)

In [ ]:
len_embed = len(df_embed['text_embed'].iloc[0])
len_embed

In [ ]:
df_embed.to_excel(f"./xlsx/{video_id}_embed.xlsx", sheet_name=video_id, index=False)

In [ ]:
search_term = 'yearly return without fees'
search_term

In [ ]:
search_term_embed = ollama_emb.embed_query(search_term)
search_term_embed

In [ ]:
len_search_term = len(search_term_embed)
len_search_term

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
df_embed.loc[:, 'cosine_similarity'] = df_embed['text_embed'].apply(lambda x: cosine_similarity(x, search_term_embed))

In [ ]:
df_embed.head(5)

In [ ]:
df_sorted = df_embed.sort_values(by='cosine_similarity', ascending=False)
df_sorted

In [ ]:
jump_to_location = df_sorted['watch_url'].iloc[0]
jump_to_location